In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

In [13]:
#creating a dataframe to do prediction
np.random.seed(42)#to fix the value for each execution
data = pd.DataFrame({
    'voltage': np.random.normal(230, 10, 1000),# 1000- random values,230-mean ,10 standard deviation
    'current': np.random.normal(10, 2, 1000),
    'temperature': np.random.normal(75, 10, 1000),
    'vibration': np.random.normal(0.5, 0.2, 1000),
    'power_factor': np.random.normal(0.95, 0.02, 1000)
})

In [14]:
#creating new field failure
# Simulate failures based on thresholds
# Simulate failures based on thresholds
data['failure'] = np.where(
    (data['temperature'] > 90) |
    (data['vibration'] > 0.8) |
    (data['power_factor'] < 0.90) |
    (data['voltage'] < 200) |       # Example voltage condition (too low)
    (data['current'] > 10),         # Example current condition (too high)
    1,  # Failure
    0   # Normal
)


In [15]:
#Split and preprocess the data
X = data.drop('failure', axis=1)#except failure column we take other column value  for x
y = data['failure']#failure column taken to get predicted

scaler = StandardScaler()# removing the mean and scaling to unit variance that is 0 and 1
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [16]:
# Train a Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)#many decision trees and combines their outputs to improve performance and give accurate performance.it takes number of parameters as input
#and based on that it will draw decision tree
model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [17]:
#testing the model with our  generated data set and its prediction accuracy
y_pred = model.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        89
           1       1.00      1.00      1.00       111

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200

Confusion Matrix:
 [[ 89   0]
 [  0 111]]


In [18]:
def trigger_maintenance_alert(sensor_data, model, scaler, feature_names):
    """
    Predict failure and explain cause based on sensor input.

    Args:
        sensor_data (list or array): Real-time sensor input in the same order as training.
        model: Trained model (with feature_importances_ attribute).
        scaler: Scaler used during model training.
        feature_names (list): List of feature names in order.

    Returns:
        str: Explanation message of system status or failure reason.
    """
    import numpy as np

    # Scale the input data
    sensor_scaled = scaler.transform([sensor_data])

    # Predict failure (assuming binary classification: 0 = OK, 1 = Failure)
    prediction = model.predict(sensor_scaled)[0]

    # Analyze feature contributions
    importances = model.feature_importances_
    contribution_scores = sensor_scaled[0] * importances

    top_feature_index = int(np.argmax(contribution_scores))
    top_feature_name = feature_names[top_feature_index] if top_feature_index < len(feature_names) else "unknown"

    # Map feature names to user-friendly descriptions
    feature_names_map = {
        "voltage": "abnormal voltage (too low or high)",
        "current": "abnormal current flow",
        "temperature": "excessive temperature",
        "vibration": "high vibration levels",
        "power_factor": "poor power factor (inefficient usage)"
    }

    if prediction == 1:
        cause_description = feature_names_map.get(top_feature_name, "sensor anomaly or unknown issue")
        return (
            f"⚠️ **Maintenance Alert:**\n"
            f"Early sign of system failure detected.\n"
            f"🚨 Likely cause: **{cause_description}** (feature: `{top_feature_name}`)."
        )
    else:
        return "✅ All systems are operating normally."


In [19]:
sensor_data = [220, 4.5, 70, 0.05, 0.95]
# voltage = 220      ✅ OK
# current = 4.5      ✅ OK
# temperature = 70   ✅ OK (<90)
# vibration = 0.05   ✅ OK (<0.8)
# power_factor = 0.95 ✅ OK (>= 0.90)
feature_names = ["voltage","current","temperature","ibration","power_factor"]

message = trigger_maintenance_alert(sensor_data, model, scaler, feature_names)
print(message)


✅ All systems are operating normally.


C:\Users\Welcome\AppData\Roaming\Python\Python313\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [20]:
sensor_data = [190, 4.8, 95, 0.6, 0.92]
# temperature = 95  ❌ (>90)
feature_names = ["voltage","current","temperature","ibration","power_factor"]
message = trigger_maintenance_alert(sensor_data, model, scaler, feature_names)
print(message)


⚠️ **Maintenance Alert:**
Early sign of system failure detected.
🚨 Likely cause: **excessive temperature** (feature: `temperature`).


C:\Users\Welcome\AppData\Roaming\Python\Python313\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [21]:
sensor_data = [210, 5.0, 80, 0.9, 0.85]
# vibration = 0.9 ❌
# power_factor = 0.85 ❌
feature_names = ["voltage","current","temperature","ibration","power_factor"]
message = trigger_maintenance_alert(sensor_data, model, scaler, feature_names)
print(message)


⚠️ **Maintenance Alert:**
Early sign of system failure detected.
🚨 Likely cause: **sensor anomaly or unknown issue** (feature: `ibration`).


C:\Users\Welcome\AppData\Roaming\Python\Python313\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [22]:
import joblib

joblib.dump(model, "model.pkl")
joblib.dump(scaler, "scaler.pkl")


['scaler.pkl']

In [23]:
pip install flask joblib


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
